<a href="https://colab.research.google.com/github/ayandalab/NLP_Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [59]:
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import pandas as pd # Data analysis and manipulation tool

# to ignore warnings
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
train_data = pd.read_csv('/content/gdrive/MyDrive/Train_Data.csv')

In [61]:
train_data.head()

,review,sentiment
0,think robert ryans best film portrayed someone...,1
1,juano hernandez exceptional actor played suppo...,1
2,shocked sign indicate cash,0
3,sat another ten minute finally gave left,0
4,igo charger tip really great,1


In [62]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060 entries, 0 to 2059
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     2060 non-null   object
 1   sentiment  2060 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 32.3+ KB


In [63]:
train_data['review'] = train_data['review'].str.lower()


def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

train_data['review'] = train_data['review'].apply(remove_special_characters, remove_digits=False)

def f(r):
    wnl = WordNetLemmatizer()
    words = nltk.word_tokenize(r)
    lemmatized_words = [wnl.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    return " ".join(lemmatized_words)

train_data['review'] = train_data['review'].apply(f)

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data_X = cv.fit_transform(train_data['review']).toarray()
data_X = pd.DataFrame(data_X, columns=cv.get_feature_names())

In [65]:
X = data_X
y = train_data.sentiment

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.002, random_state = 42)

In [67]:
# Importing LogisticRegression from sklearn.linear_model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
# import mean squared error from sklearn.metric
from sklearn.metrics import accuracy_score
print('Accuracy Score is: ', accuracy_score(y_val, pred)*100)

Accuracy Score is:  100.0


In [68]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_val)
print(accuracy_score(y_val, y_pred)*100)

80.0


In [69]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=25, random_state=75)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_val)
print(accuracy_score(y_val, y_pred)*100)

60.0


In [70]:
test_data = pd.read_csv('/content/gdrive/MyDrive/Test_Data.csv')

In [71]:
test_data.head()

,review
0,great service food
1,pairing iphone could happier far
2,plot hole pair fishnet stocking direction edit...
3,surely doesnt know make coherent action movie ...
4,still quite interesting entertaining follow


In [72]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  686 non-null    object
dtypes: object(1)
memory usage: 5.5+ KB


In [73]:
test_data['review'] = test_data['review'].str.lower()


def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

test_data['review'] = test_data['review'].apply(remove_special_characters, remove_digits=False)

def f(r):
    wnl = WordNetLemmatizer()
    words = nltk.word_tokenize(r)
    lemmatized_words = [wnl.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    return " ".join(lemmatized_words)

test_data['review'] = test_data['review'].apply(f)

In [74]:
test = cv.transform(test_data['review']).toarray()
test_data = pd.DataFrame(test, columns=cv.get_feature_names())

In [75]:
target = lr.predict(test_data)

In [76]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(target) # target are nothing but the final predictions of your model on input features of your new unseen test data
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('submission.csv', index = False)
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>